## Visualizations of NYC POI Data

### Inner Join

In [117]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as spstats
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler

In [135]:
# Load POI datasets
gp = pd.read_csv('./clean_data/gp_poi_clean.csv')
#yelp = pd.read_csv('./clean_data/yelp_poi_cleaned.csv')
#fsq = pd.read_csv('./clean_data/fsq_poi_clean.csv')
yelp = pd.read_csv('yelp_poi_cleaned.csv')
fsq = pd.read_csv('fsq_poi_clean.csv')

In [3]:
# importing borough & neighbourhood dataframes
nyc_geo = pd.read_csv('./clean_data/nyc_geo_cleaned.csv') # boroughs
nyc_bo = pd.read_csv('./clean_data/nyc_boroughs.csv') # boroughs
nyc_nb = pd.read_csv('./clean_data/nyc_neighbs.csv',encoding='cp1252') # neighbourhoods

In [136]:
nyc_nb.head()

,Port Richmond,South Beach and Tottenville,Stapleton and St. George,Mid-Island (Willowbrook),Inwood and Washington Heights,Central Harlem,East Harlem,Upper West Side,Upper East Side,Chelsea and Clinton,...,Northwest Brooklyn,Central Brooklyn (Bedford Stuyvesant – Crown Heights),East New York and New Lots,Sunset Park,Southwest Brooklyn,Borough Park,Canarsie and Flatlands,Southern Brooklyn,Flatbush,Bushwick and Williamsburg
0,10302.0,10306.0,10301.0,10314.0,10031.0,10026.0,10029.0,10023.0,10021.0,10001.0,...,11201.0,11212.0,11207.0,11220.0,11209.0,11204.0,11234.0,11223.0,11203.0,11206.0
1,10303.0,10307.0,10304.0,NaN,10032.0,10027.0,10035.0,10024.0,10028.0,10011.0,...,11205.0,11213.0,11208.0,11232.0,11214.0,11218.0,11236.0,11224.0,11210.0,11221.0
2,10310.0,10308.0,10305.0,NaN,10033.0,10030.0,NaN,10025.0,10044.0,10018.0,...,11215.0,11216.0,NaN,NaN,11228.0,11219.0,11239.0,11229.0,11225.0,11237.0
3,NaN,NaN,NaN,NaN,10034.0,10037.0,NaN,NaN,10065.0,10019.0,...,11217.0,11233.0,NaN,NaN,NaN,11230.0,NaN,11235.0,11226.0,NaN
4,NaN,NaN,NaN,NaN,10040.0,10039.0,NaN,NaN,10075.0,10020.0,...,11231.0,11238.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
nyc_bo.head()

,Manhattan,Staten Island,Bronx,Queens,Brooklyn
0,10001.0,10301.0,10451.0,11004,11201.0
1,10002.0,10302.0,10452.0,11005,11202.0
2,10003.0,10303.0,10453.0,11006,11203.0
3,10004.0,10304.0,10454.0,11007,11204.0
4,10005.0,10305.0,10455.0,11008,11205.0


In [138]:
yelp.head()

,Unnamed: 0,Name,Reviews,Rating,Price,Distance (m),Latitude,Longitude,Address,Borough,Zipcode,Category
0,0,Ripe Kitchen & Bar,287,4.0,$$,805.077589,40.898214,-73.838821,151 W Sandford Blvd,Mount Vernon,10550,restaurant
1,1,H.I.M Ital Health Food Market,47,4.5,$,715.467929,40.897633,-73.854665,4374B White Plains Rd,Bronx,10466,restaurant
2,2,Ali's Roti Shop,104,4.0,$,809.642747,40.894000,-73.856840,4220 White Plains Rd,Bronx,10466,restaurant
3,3,Paula's Soul Cafe,202,3.0,$$,746.891106,40.892690,-73.855680,746 E 233rd St,Bronx,10466,restaurant
4,4,Jerk House,62,3.5,$$,771.525854,40.894640,-73.856480,4246 White Plains,Bronx,10466,restaurant


In [139]:
yelp.drop(columns=['Unnamed: 0','Address'], inplace=True)

In [140]:
# need to change price to number scale
yelp['Price'].unique()

array(['$$', '$', '$$$', nan, '$$$$'], dtype=object)

In [141]:
# looks like scale is 1-4
yelp['Price'].replace({'$':1, '$$' :2, '$$$':3, '$$$$':4}, inplace=True)

In [142]:
yelp['Price'].unique()

array([ 2.,  1.,  3., nan,  4.])

In [143]:
# there are still some nans, but theoretically some will get fixed once we merge

In [144]:
fsq.head()

,Unnamed: 0,Distance (m),Name,Price,Rating,Borough,Neighborhood,Zipcode,Latitude,Longitude,Category
0,0,127.0,Lollipops Gelato,1.0,8.6,Bronx,Edenwald,10466.0,40.894705,-73.847201,restaurant
1,1,797.0,Ripe Kitchen and Bar,2.0,8.4,Mount Vernon,Wakefield,10550.0,40.894705,-73.847201,restaurant
2,2,821.0,Ali's Roti Shop,1.0,8.1,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant
3,3,983.0,Jimbo's Hamburger Palace,1.0,8.0,Bronx,Williambridge,10466.0,40.894705,-73.847201,restaurant
4,4,454.0,Cooler Runnings Jamaican Restaurant,2.0,6.4,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant


In [145]:
fsq.rename(columns={'Neighborhood':'Neighbourhood'}, inplace=True)
fsq.drop(columns='Unnamed: 0', inplace=True)

In [146]:
fsq.head()

,Distance (m),Name,Price,Rating,Borough,Neighbourhood,Zipcode,Latitude,Longitude,Category
0,127.0,Lollipops Gelato,1.0,8.6,Bronx,Edenwald,10466.0,40.894705,-73.847201,restaurant
1,797.0,Ripe Kitchen and Bar,2.0,8.4,Mount Vernon,Wakefield,10550.0,40.894705,-73.847201,restaurant
2,821.0,Ali's Roti Shop,1.0,8.1,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant
3,983.0,Jimbo's Hamburger Palace,1.0,8.0,Bronx,Williambridge,10466.0,40.894705,-73.847201,restaurant
4,454.0,Cooler Runnings Jamaican Restaurant,2.0,6.4,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant


In [147]:
gp.head()

,Name,Price,Rating,Reviews,Borough,Neighborhood,Address,Zipcode,Latitude,Longitude,Category
0,New China Garden,1.0,3.9,85.0,Bronx,Woodlawn,"724 Nereid Avenue, The Bronx",10466.0,40.897919,-73.853364,restaurant
1,El Jobo,NaN,3.3,216.0,Bronx,Woodlawn,"748 East 233rd Street, The Bronx",10466.0,40.892678,-73.855632,restaurant
2,Paula's,2.0,3.9,697.0,Bronx,Woodlawn,"746 East 233rd Street, The Bronx",10466.0,40.892692,-73.855678,restaurant
3,Jerk House Caribbean Restaurant,NaN,4.2,429.0,Bronx,Woodlawn,"4246 White Plains Road, The Bronx",10466.0,40.894663,-73.856400,restaurant
4,McDonald's,1.0,3.7,653.0,Bronx,Woodlawn,"4174 White Plains Road, The Bronx",10466.0,40.892779,-73.857473,restaurant


In [148]:
gp.rename(columns={'Neighborhood':'Neighbourhood'}, inplace=True)

In [149]:
print('Price:')
print(fsq['Price'].describe())
print('Rating:')
print(fsq['Rating'].describe())

Price:
count    6269.000000
mean        1.594353
std         0.704104
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         4.000000
Name: Price, dtype: float64
Rating:
count    4685.000000
mean        7.803308
std         1.004244
min         4.500000
25%         7.100000
50%         8.000000
75%         8.700000
max         9.600000
Name: Rating, dtype: float64


In [150]:
print('Price:')
print(yelp['Price'].describe())
print('Rating:')
print(yelp['Rating'].describe())

Price:
count    6116.000000
mean        1.797907
std         0.629113
min         1.000000
25%         1.000000
50%         2.000000
75%         2.000000
max         4.000000
Name: Price, dtype: float64
Rating:
count    7115.000000
mean        3.916936
std         0.630598
min         1.000000
25%         3.500000
50%         4.000000
75%         4.500000
max         5.000000
Name: Rating, dtype: float64


In [151]:
print('Price:')
print(gp['Price'].describe())
print('Rating:')
print(gp['Rating'].describe())

Price:
count    3583.000000
mean        1.585822
std         0.628150
min         0.000000
25%         1.000000
50%         2.000000
75%         2.000000
max         4.000000
Name: Price, dtype: float64
Rating:
count    10862.000000
mean         4.129507
std          0.673147
min          1.000000
25%          3.900000
50%          4.200000
75%          4.500000
max          5.000000
Name: Rating, dtype: float64


In [152]:
# setting range for rating to be 1-5 since prices are all already on a scale of 1-4
scaler = MinMaxScaler((1,5))

# Rescale rating
fsq['Rating'] = pd.DataFrame(scaler.fit_transform(fsq['Rating'].values.reshape(-1,1)))
yelp['Rating'] = pd.DataFrame(scaler.fit_transform(yelp['Rating'].values.reshape(-1,1)))
gp['Rating'] = pd.DataFrame(scaler.fit_transform(gp['Rating'].values.reshape(-1,1)))


In [153]:
print('Foursquare Shape:', fsq.shape)
print('Yelp Shape:', yelp.shape)
print('Google Places Shape:', gp.shape)

Foursquare Shape: (7001, 10)
Yelp Shape: (7115, 10)
Google Places Shape: (15517, 11)


### Foursquare API has latitude & longitude that matches our nyc_geo file so we will fill in any NaN's using that

In [154]:
nyc_geo.head()

,Borough,Neighbourhood,Latitude,Longitude,ll,Zipcode,ll_round
0,Bronx,Wakefield,40.894705,-73.847201,"40.89470517661,-73.84720052054902",10466.0,"40.895,-73.847"
1,Bronx,Co-op City,40.874294,-73.829939,"40.87429419303012,-73.82993910812398",10475.0,"40.874,-73.83"
2,Bronx,Eastchester,40.887556,-73.827806,"40.887555677350775,-73.82780644716412",10466.0,"40.888,-73.828"
3,Bronx,Fieldston,40.895437,-73.905643,"40.89543742690383,-73.90564259591682",10471.0,"40.895,-73.906"
4,Bronx,Riverdale,40.890834,-73.912585,"40.890834493891305,-73.9125854610857",10463.0,"40.891,-73.913"


In [155]:
# assigning a lat/lon to search on
fsq['ll'] = fsq['Latitude'].astype(str) + ',' + fsq['Longitude'].astype(str)
nyc_geo['ll'] = nyc_geo['Latitude'].astype(str) + ',' + nyc_geo['Longitude'].astype(str)

In [156]:
nyc_geo

,Borough,Neighbourhood,Latitude,Longitude,ll,Zipcode,ll_round
0,Bronx,Wakefield,40.894705,-73.847201,"40.89470517661,-73.84720052054902",10466.0,"40.895,-73.847"
1,Bronx,Co-op City,40.874294,-73.829939,"40.87429419303012,-73.82993910812398",10475.0,"40.874,-73.83"
2,Bronx,Eastchester,40.887556,-73.827806,"40.887555677350775,-73.82780644716412",10466.0,"40.888,-73.828"
3,Bronx,Fieldston,40.895437,-73.905643,"40.89543742690383,-73.90564259591682",10471.0,"40.895,-73.906"
4,Bronx,Riverdale,40.890834,-73.912585,"40.890834493891305,-73.9125854610857",10463.0,"40.891,-73.913"
...,...,...,...,...,...,...,...
301,Manhattan,Hudson Yards,40.756658,-74.000111,"40.75665808227519,-74.00011136202637",10001.0,"40.757,-74.0"
302,Queens,Hammels,40.587338,-73.805530,"40.58733774018741,-73.80553002968718",11693.0,"40.587,-73.806"
303,Queens,Bayswater,40.611322,-73.765968,"40.61132169128384,-73.76596781445627",11691.0,"40.611,-73.766"
304,Queens,Queensbridge,40.756091,-73.945631,"40.75609129709471,-73.94563070334091",11101.0,"40.756,-73.946"


In [157]:
fsq

,Distance (m),Name,Price,Rating,Borough,Neighbourhood,Zipcode,Latitude,Longitude,Category,ll
0,127.0,Lollipops Gelato,1.0,4.215686,Bronx,Edenwald,10466.0,40.894705,-73.847201,restaurant,"40.89470517661,-73.84720052054902"
1,797.0,Ripe Kitchen and Bar,2.0,4.058824,Mount Vernon,Wakefield,10550.0,40.894705,-73.847201,restaurant,"40.89470517661,-73.84720052054902"
2,821.0,Ali's Roti Shop,1.0,3.823529,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant,"40.89470517661,-73.84720052054902"
3,983.0,Jimbo's Hamburger Palace,1.0,3.745098,Bronx,Williambridge,10466.0,40.894705,-73.847201,restaurant,"40.89470517661,-73.84720052054902"
4,454.0,Cooler Runnings Jamaican Restaurant,2.0,2.490196,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant,"40.89470517661,-73.84720052054902"
...,...,...,...,...,...,...,...,...,...,...,...
6996,410.0,My Way Deli,NaN,NaN,Staten Island,Stapleton,10304.0,40.617311,-74.081740,restaurant,"40.61731079252983,-74.08173992211962"
6997,821.0,Campo Bello,NaN,NaN,Staten Island,Stapleton,10304.0,40.617311,-74.081740,restaurant,"40.61731079252983,-74.08173992211962"
6998,826.0,Al Baraka Restaurant,NaN,NaN,Staten Island,Stapleton,10304.0,40.617311,-74.081740,restaurant,"40.61731079252983,-74.08173992211962"
6999,836.0,Chicken R Us,NaN,NaN,Staten Island,Stapleton,10304.0,40.617311,-74.081740,restaurant,"40.61731079252983,-74.08173992211962"


In [158]:
"""
Loops through the nyc_geo (nyc boroughs) dataframe to see if one of its coordinates
is within the coordinates string of our addresses dataframe. Then fills in neighbourhood & borough names
"""

num_rows = nyc_geo.shape[0]
row_count = 0

while row_count < num_rows:
    fsq.loc[fsq['ll'] ==nyc_geo['ll'][row_count], 'Neighbourhood'] = nyc_geo['Neighbourhood'][row_count]
    fsq.loc[fsq['ll'] ==nyc_geo['ll'][row_count], 'Borough'] = nyc_geo['Borough'][row_count]
    row_count +=1

In [159]:
nyc_geo['Zipcode'].unique()

array([10466., 10475., 10471., 10463., 10034., 10458., 10467., 10461.,
       10464., 10453., 10468., 10457., 10460., 10452., 10455., 10454.,
       10459., 10474., 10473., 10465., 10462., 10469., 10472., 11209.,
       11214., 11220., 11222., 11223., 11235., 11230., 11218., 11225.,
       11203., 11238., 11212., 11211., 11221., 11216., 11201., 11231.,
       11215., 11205., 11208., 11207., 11236., 11234., 11224., 11219.,
       11217., 11233., 11416., 11206., 11249., 10002., 10032., 10039.,
       10031., 10030., 10035., 10028., 10075., 11106., 10024., 10023.,
       10018., 10017., 10016., 10014., 10003., 10009., 10012., 10013.,
       10025., 10027., 10010., 10038., 11105., 11372., 11373., 11414.,
       11375., 11415., 11418., 11354., 11101., 11104., 11368., 11378.,
       11385., 11421., 11417., 11420., 11356., 11357., 11361., 11358.,
       11362., 11004., 11426., 11367., 11366., 11435., 11432., 11364.,
       11429., 11423., 11412., 11434., 11411., 11422., 11691., 11693.,
      

In [160]:
"""
Assigning zipcodes to every neighbourhood, based on the reverse of the above
"""

num_rows = fsq.shape[0]
row_count = 0

while row_count < num_rows:
    nyc_geo.loc[nyc_geo['ll'] ==fsq['ll'][row_count], 'Zipcode'] = fsq['Zipcode'][row_count]

    row_count +=1

In [161]:
# rounding coordinates in hopes of getting a better match

In [162]:
nyc_geo['ll_round'] = nyc_geo['Latitude'].round(3).astype(str) + ',' + nyc_geo['Longitude'].round(3).astype(str)

In [163]:
yelp['ll'] = yelp['ll'] = yelp['Latitude'].round(3).astype(str) + ',' + yelp['Longitude'].round(3).astype(str)

In [164]:
# yelp json did not have our original latlon coordinates, so we will be matching against zipcodes as well
# not a perfect solution, but usable
num_rows = nyc_geo.shape[0]
row_count = 0

while row_count < num_rows:
    yelp.loc[yelp['Zipcode'] ==nyc_geo['Zipcode'][row_count], 'Neighbourhood'] = nyc_geo['Neighbourhood'][row_count]
    yelp.loc[yelp['ll'] ==nyc_geo['ll_round'][row_count], 'Neighbourhood'] = nyc_geo['Neighbourhood'][row_count]
    yelp.loc[yelp['Zipcode'] ==nyc_geo['Zipcode'][row_count], 'Borough'] = nyc_geo['Borough'][row_count]
    yelp.loc[yelp['ll'] ==nyc_geo['ll_round'][row_count], 'Borough'] = nyc_geo['Borough'][row_count]
    row_count +=1

In [165]:
yelp

,Name,Reviews,Rating,Price,Distance (m),Latitude,Longitude,Borough,Zipcode,Category,ll,Neighbourhood
0,Ripe Kitchen & Bar,287,4.0,2.0,805.077589,40.898214,-73.838821,Mount Vernon,10550,restaurant,"40.898,-73.839",NaN
1,H.I.M Ital Health Food Market,47,4.5,1.0,715.467929,40.897633,-73.854665,Bronx,10466,restaurant,"40.898,-73.855",Woodlawn
2,Ali's Roti Shop,104,4.0,1.0,809.642747,40.894000,-73.856840,Bronx,10466,restaurant,"40.894,-73.857",Woodlawn
3,Paula's Soul Cafe,202,3.0,2.0,746.891106,40.892690,-73.855680,Bronx,10466,restaurant,"40.893,-73.856",Woodlawn
4,Jerk House,62,3.5,2.0,771.525854,40.894640,-73.856480,Bronx,10466,restaurant,"40.895,-73.856",Woodlawn
...,...,...,...,...,...,...,...,...,...,...,...,...
7110,13 Golden Deli,1,5.0,NaN,788.768929,40.624331,-74.079396,Staten Island,10304,restaurant,"40.624,-74.079",Fox Hills
7111,Al Baraka Restaurant,4,3.5,NaN,825.929209,40.611660,-74.088120,Staten Island,10304,restaurant,"40.612,-74.088",Fox Hills
7112,Frank's Bay Pizza,5,2.5,NaN,860.455769,40.624840,-74.079340,Staten Island,10304,restaurant,"40.625,-74.079",Fox Hills
7113,Mexico Azteca,2,4.5,1.0,1049.847125,40.625879,-74.076515,Staten Island,10304,restaurant,"40.626,-74.077",Fox Hills


In [166]:
gp['ll'] = gp['ll'] = gp['Latitude'].round(3).astype(str) + ',' + gp['Longitude'].round(3).astype(str)

In [167]:
num_rows = nyc_geo.shape[0]
row_count = 0

while row_count < num_rows:
    gp.loc[gp['Zipcode'] ==nyc_geo['Zipcode'][row_count], 'Neighbourhood'] = nyc_geo['Neighbourhood'][row_count]
    gp.loc[gp['ll'] ==nyc_geo['ll_round'][row_count], 'Neighbourhood'] = nyc_geo['Neighbourhood'][row_count]
    gp.loc[gp['Zipcode'] ==nyc_geo['Zipcode'][row_count], 'Borough'] = nyc_geo['Borough'][row_count]
    gp.loc[gp['ll'] ==nyc_geo['ll_round'][row_count], 'Borough'] = nyc_geo['Borough'][row_count]
    row_count +=1

In [168]:
gp

,Name,Price,Rating,Reviews,Borough,Neighbourhood,Address,Zipcode,Latitude,Longitude,Category,ll
0,New China Garden,1.0,3.9,85.0,Bronx,Woodlawn,"724 Nereid Avenue, The Bronx",10466.0,40.897919,-73.853364,restaurant,"40.898,-73.853"
1,El Jobo,NaN,3.3,216.0,Bronx,Woodlawn,"748 East 233rd Street, The Bronx",10466.0,40.892678,-73.855632,restaurant,"40.893,-73.856"
2,Paula's,2.0,3.9,697.0,Bronx,Woodlawn,"746 East 233rd Street, The Bronx",10466.0,40.892692,-73.855678,restaurant,"40.893,-73.856"
3,Jerk House Caribbean Restaurant,NaN,4.2,429.0,Bronx,Woodlawn,"4246 White Plains Road, The Bronx",10466.0,40.894663,-73.856400,restaurant,"40.895,-73.856"
4,McDonald's,1.0,3.7,653.0,Bronx,Woodlawn,"4174 White Plains Road, The Bronx",10466.0,40.892779,-73.857473,restaurant,"40.893,-73.857"
...,...,...,...,...,...,...,...,...,...,...,...,...
15512,Foodtown of Roosevelt Island,NaN,3.9,186.0,Manhattan,Upper East Side,"686 Main Street, New York",10044.0,40.764702,-73.946986,supermarket,"40.765,-73.947"
15513,Family Dollar,1.0,3.9,187.0,Staten Island,Concord,"375 Tompkins Avenue, Staten Island",10305.0,40.616945,-74.073431,supermarket,"40.617,-74.073"
15514,Stop 1 Supermarket,NaN,4.5,64.0,Staten Island,Fox Hills,"515-517 Targee Street, Staten Island",10304.0,40.614688,-74.084539,supermarket,"40.615,-74.085"
15515,Targee African Grocery Inc,NaN,4.2,19.0,Staten Island,Fox Hills,"523 Targee Street, Staten Island",10304.0,40.614483,-74.084457,supermarket,"40.614,-74.084"


In [169]:
# Merging Dataframes

In [170]:
poi_merge = fsq.merge(gp, how='outer', on=('Name','Neighbourhood'))

In [171]:
poi_merge = poi_merge.merge(yelp, how='outer', on=('Name','Neighbourhood'))

In [172]:
poi_merge.head()

,Distance (m)_x,Name,Price_x,Rating_x,Borough_x,Neighbourhood,Zipcode_x,Latitude_x,Longitude_x,Category_x,...,Reviews_y,Rating,Price,Distance (m)_y,Latitude,Longitude,Borough,Zipcode,Category,ll
0,127.0,Lollipops Gelato,1.0,4.215686,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,797.0,Ripe Kitchen and Bar,2.0,4.058824,Bronx,Wakefield,10550.0,40.894705,-73.847201,restaurant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,821.0,Ali's Roti Shop,1.0,3.823529,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,983.0,Jimbo's Hamburger Palace,1.0,3.745098,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,454.0,Cooler Runnings Jamaican Restaurant,2.0,2.490196,Bronx,Wakefield,10466.0,40.894705,-73.847201,restaurant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
poi_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27875 entries, 0 to 27874
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Distance (m)_x  7469 non-null   float64
 1   Name            27873 non-null  object 
 2   Price_x         6704 non-null   float64
 3   Rating_x        5051 non-null   float64
 4   Borough_x       7471 non-null   object 
 5   Neighbourhood   27119 non-null  object 
 6   Zipcode_x       7466 non-null   float64
 7   Latitude_x      7471 non-null   float64
 8   Longitude_x     7471 non-null   float64
 9   Category_x      7471 non-null   object 
 10  ll_x            7471 non-null   object 
 11  Price_y         4186 non-null   float64
 12  Rating_y        11531 non-null  float64
 13  Reviews_x       11531 non-null  float64
 14  Borough_y       16186 non-null  object 
 15  Address         16184 non-null  object 
 16  Zipcode_y       16184 non-null  float64
 17  Latitude_y      16186 non-null 

In [174]:
poi_merge['Price'].fillna(poi_merge['Price_x'], inplace=True)
poi_merge['Price'].fillna(poi_merge['Price_y'], inplace=True)
poi_merge['Rating'].fillna(poi_merge['Rating_x'], inplace=True)
poi_merge['Rating'].fillna(poi_merge['Rating_y'], inplace=True)
poi_merge['Reviews_x'].fillna(poi_merge['Reviews_y'], inplace=True)
poi_merge['Borough'].fillna(poi_merge['Borough_x'], inplace=True)
poi_merge['Borough'].fillna(poi_merge['Borough_y'], inplace=True)
poi_merge['Latitude'].fillna(poi_merge['Latitude_x'], inplace=True)
poi_merge['Latitude'].fillna(poi_merge['Latitude_y'], inplace=True)
poi_merge['Longitude'].fillna(poi_merge['Longitude_x'], inplace=True)
poi_merge['Longitude'].fillna(poi_merge['Longitude_y'], inplace=True)
poi_merge['Category'].fillna(poi_merge['Category_x'], inplace=True)
poi_merge['Category'].fillna(poi_merge['Category_y'], inplace=True)
poi_merge['Distance (m)_x'].fillna(poi_merge['Distance (m)_y'], inplace=True)

In [175]:
poi_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27875 entries, 0 to 27874
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Distance (m)_x  13552 non-null  float64
 1   Name            27873 non-null  object 
 2   Price_x         6704 non-null   float64
 3   Rating_x        5051 non-null   float64
 4   Borough_x       7471 non-null   object 
 5   Neighbourhood   27119 non-null  object 
 6   Zipcode_x       7466 non-null   float64
 7   Latitude_x      7471 non-null   float64
 8   Longitude_x     7471 non-null   float64
 9   Category_x      7471 non-null   object 
 10  ll_x            7471 non-null   object 
 11  Price_y         4186 non-null   float64
 12  Rating_y        11531 non-null  float64
 13  Reviews_x       17095 non-null  float64
 14  Borough_y       16186 non-null  object 
 15  Address         16184 non-null  object 
 16  Zipcode_y       16184 non-null  float64
 17  Latitude_y      16186 non-null 

In [176]:
poi_merge.drop(columns=['Address','Price_x', 'Price_y', 'Rating_x', 'Rating_y', 'Reviews_y', 'Borough_x', 'Borough_y',
                        'Latitude_x','Latitude_y','Longitude_x','Longitude_y',
                        'Category_x', 'Category_y', 'Distance (m)_y','Zipcode_x','Zipcode_y','ll_x','ll_y','ll'], inplace=True)

In [177]:
# filling in NaN zipcodes
num_rows = nyc_geo.shape[0]
row_count = 0

while row_count < num_rows:
    poi_merge.loc[poi_merge['Neighbourhood'] ==nyc_geo['Neighbourhood'][row_count], 'Zipcode'] = nyc_geo['Zipcode'][row_count]
    row_count +=1

In [178]:
poi_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27875 entries, 0 to 27874
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Distance (m)_x  13552 non-null  float64
 1   Name            27873 non-null  object 
 2   Neighbourhood   27119 non-null  object 
 3   Reviews_x       17095 non-null  float64
 4   Rating          21107 non-null  float64
 5   Price           14495 non-null  float64
 6   Latitude        27875 non-null  float64
 7   Longitude       27875 non-null  float64
 8   Borough         27875 non-null  object 
 9   Zipcode         26764 non-null  float64
 10  Category        27875 non-null  object 
dtypes: float64(7), object(4)
memory usage: 2.6+ MB


In [179]:
poi_merge.rename(columns={'Distance (m)_x':'Distance (m)', 'Reviews_x' :'Reviews'}, inplace=True)

In [180]:
poi_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27875 entries, 0 to 27874
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Distance (m)   13552 non-null  float64
 1   Name           27873 non-null  object 
 2   Neighbourhood  27119 non-null  object 
 3   Reviews        17095 non-null  float64
 4   Rating         21107 non-null  float64
 5   Price          14495 non-null  float64
 6   Latitude       27875 non-null  float64
 7   Longitude      27875 non-null  float64
 8   Borough        27875 non-null  object 
 9   Zipcode        26764 non-null  float64
 10  Category       27875 non-null  object 
dtypes: float64(7), object(4)
memory usage: 2.6+ MB


In [181]:
poi_merge.groupby('Category').count()

,Distance (m),Name,Neighbourhood,Reviews,Rating,Price,Latitude,Longitude,Borough,Zipcode
Category,,,,,,,,,,
park,0,2294,2276,1905,1905,11,2294,2294,2294,2086
restaurant,13552,16367,15684,10161,14173,14270,16369,16369,16369,16096
school,0,3572,3546,2342,2342,3,3572,3572,3572,3318
supermarket,0,1465,1459,1382,1382,210,1465,1465,1465,1336
transit_station,0,4175,4154,1305,1305,1,4175,4175,4175,3928


In [182]:
poi_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27875 entries, 0 to 27874
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Distance (m)   13552 non-null  float64
 1   Name           27873 non-null  object 
 2   Neighbourhood  27119 non-null  object 
 3   Reviews        17095 non-null  float64
 4   Rating         21107 non-null  float64
 5   Price          14495 non-null  float64
 6   Latitude       27875 non-null  float64
 7   Longitude      27875 non-null  float64
 8   Borough        27875 non-null  object 
 9   Zipcode        26764 non-null  float64
 10  Category       27875 non-null  object 
dtypes: float64(7), object(4)
memory usage: 2.6+ MB


In [183]:
# dropping duplicates where name, neighbourhood, and category are all the same
len(poi_merge[poi_merge.duplicated(subset=['Name','Neighbourhood', 'Category'])])

3075

In [184]:
poi_merge.drop_duplicates(subset=['Name','Neighbourhood', 'Category'], inplace=True)

In [185]:
# looking for null neighbourhoods after running above algorithm
len(poi_merge[poi_merge['Neighbourhood'].isna()])

542

In [186]:
# not a big number, so we can just drop them
poi_merge.dropna(subset='Neighbourhood',inplace=True)

In [187]:
poi_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24258 entries, 0 to 27874
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Distance (m)   10291 non-null  float64
 1   Name           24256 non-null  object 
 2   Neighbourhood  24258 non-null  object 
 3   Reviews        13581 non-null  float64
 4   Rating         17566 non-null  float64
 5   Price          11408 non-null  float64
 6   Latitude       24258 non-null  float64
 7   Longitude      24258 non-null  float64
 8   Borough        24258 non-null  object 
 9   Zipcode        23256 non-null  float64
 10  Category       24258 non-null  object 
dtypes: float64(7), object(4)
memory usage: 2.2+ MB


In [188]:
# Drop unused rows that are not part of any borough
poi_merge.drop(poi_merge[poi_merge['Borough'] == 'Mount Vernon'].index, inplace=True)
poi_merge.drop(poi_merge[poi_merge['Borough'] == 'Pelham Manor'].index, inplace=True)
poi_merge.drop(poi_merge[poi_merge['Borough'] == 'Pelham'].index, inplace=True)
poi_merge.drop(poi_merge[poi_merge['Borough'] == 'Yonkers'].index, inplace=True)
poi_merge.drop(poi_merge[poi_merge['Borough'] == 'Long Island City'].index, inplace=True)
poi_merge.drop(poi_merge[poi_merge['Borough'] == 'Floral Park'].index, inplace=True)
poi_merge.drop(poi_merge[poi_merge['Borough'] == 'Bellerose'].index, inplace=True)
poi_merge.drop(poi_merge[poi_merge['Borough'] == 'Elizabeth'].index, inplace=True)

In [191]:
# Export for cleaning
poi_merge.to_csv('merged_pois.csv', index=False)

In [189]:
poi_merge.describe()

,Distance (m),Reviews,Rating,Price,Latitude,Longitude,Zipcode
count,10291.000000,13581.000000,17566.000000,11408.000000,24258.000000,24258.000000,23256.000000
mean,621.592932,339.490833,3.945929,1.644372,40.711619,-73.935828,10814.631450
std,773.006677,1287.127744,0.746292,0.667747,0.092813,0.108992,531.973895
min,3.000000,1.000000,1.000000,0.000000,40.499251,-74.252618,7202.000000
25%,373.000000,10.000000,3.509804,1.000000,40.635325,-73.988683,10309.000000
50%,590.585302,64.000000,4.000000,2.000000,40.711454,-73.929132,11105.000000
75%,809.000000,300.000000,4.500000,2.000000,40.772563,-73.859078,11235.000000
max,23548.623893,75227.000000,5.000000,4.000000,40.912785,-73.701628,11697.000000


In [116]:
nyc_geo.to_csv('nyc_geo_latlon.csv', index=False)